In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls -lh /content/drive/MyDrive/dld_transactions


total 968M
-rw------- 1 root root 968M Jan 28 10:27 dld_transactions.csv


In [ ]:
!apt-get update -qq
!apt-get install openjdk-11-jdk-headless -qq


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!rm -f spark-3.5.0-bin-hadoop3.tgz
!wget -O spark-3.5.0-bin-hadoop3.tgz https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!ls -lh spark-3.5.0-bin-hadoop3.tgz


--2026-01-28 10:57:56--  https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz’

spark-3.5.0-bin-had 100%[===================>] 381.85M  17.5MB/s    in 25s     

2026-01-28 10:58:22 (15.0 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz’ saved [400395283/400395283]

-rw-r--r-- 1 root root 382M Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
!tar -xzf spark-3.5.0-bin-hadoop3.tgz
!ls -lh


total 382M
drwx------  5 root root 4.0K Jan 28 10:47 drive
drwxr-xr-x  1 root root 4.0K Jan 16 14:24 sample_data
drwxr-xr-x 13 1000 1000 4.0K Sep  9  2023 spark-3.5.0-bin-hadoop3
-rw-r--r--  1 root root 382M Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


Colab already has a Google package (dataproc-spark-connect) that prefers PySpark 4.x, but you’re installing PySpark 3.5.0
run this to execute that----

In [ ]:
!pip -q uninstall -y dataproc-spark-connect
!pip -q install pyspark==3.5.0



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 11.5 MB/s eta 0:00:00


In [ ]:
#  ML SETUP (clean spark init for Colab compatibility)

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DLD-ML").getOrCreate()

In [ ]:
spark

In [ ]:
csv_path = "/content/drive/MyDrive/dld_transactions/dld_transactions.csv"
csv_path


'/content/drive/MyDrive/dld_transactions/dld_transactions.csv'

In [ ]:
# Print the CSV header

!head -n 1 "$csv_path"


"transaction_id","procedure_id","trans_group_id","trans_group_ar","trans_group_en","procedure_name_ar","procedure_name_en","instance_date","property_type_id","property_type_ar","property_type_en","property_sub_type_id","property_sub_type_ar","property_sub_type_en","property_usage_ar","property_usage_en","reg_type_id","reg_type_ar","reg_type_en","area_id","area_name_ar","area_name_en","building_name_ar","building_name_en","project_number","project_name_ar","project_name_en","master_project_en","master_project_ar","nearest_landmark_ar","nearest_landmark_en","nearest_metro_ar","nearest_metro_en","nearest_mall_ar","nearest_mall_en","rooms_ar","rooms_en","has_parking","procedure_area","actual_worth","meter_sale_price","rent_value","meter_rent_price","no_of_parties_role_1","no_of_parties_role_2","no_of_parties_role_3"


In [ ]:
# Define an explicit schema
# load everything as strings first to avoid failures


from pyspark.sql.types import StructType, StructField, StringType

header_cols = open(csv_path, "r", encoding="utf-8", errors="ignore") \
    .readline() \
    .strip() \
    .split(",")

schema = StructType([StructField(col, StringType(), True) for col in header_cols])

schema


StructType([StructField('"transaction_id"', StringType(), True), StructField('"procedure_id"', StringType(), True), StructField('"trans_group_id"', StringType(), True), StructField('"trans_group_ar"', StringType(), True), StructField('"trans_group_en"', StringType(), True), StructField('"procedure_name_ar"', StringType(), True), StructField('"procedure_name_en"', StringType(), True), StructField('"instance_date"', StringType(), True), StructField('"property_type_id"', StringType(), True), StructField('"property_type_ar"', StringType(), True), StructField('"property_type_en"', StringType(), True), StructField('"property_sub_type_id"', StringType(), True), StructField('"property_sub_type_ar"', StringType(), True), StructField('"property_sub_type_en"', StringType(), True), StructField('"property_usage_ar"', StringType(), True), StructField('"property_usage_en"', StringType(), True), StructField('"reg_type_id"', StringType(), True), StructField('"reg_type_ar"', StringType(), True), StructF

In [ ]:
#Load the CSV using Spark

df_raw = spark.read.csv(
    csv_path,
    header=True,
    schema=schema,
    mode="DROPMALFORMED"
)


In [ ]:
#Validate ingestion

df_raw.printSchema()
print("Row count:", df_raw.count())
df_raw.show(5)
print("Partitions:", df_raw.rdd.getNumPartitions())


root
 |-- "transaction_id": string (nullable = true)
 |-- "procedure_id": string (nullable = true)
 |-- "trans_group_id": string (nullable = true)
 |-- "trans_group_ar": string (nullable = true)
 |-- "trans_group_en": string (nullable = true)
 |-- "procedure_name_ar": string (nullable = true)
 |-- "procedure_name_en": string (nullable = true)
 |-- "instance_date": string (nullable = true)
 |-- "property_type_id": string (nullable = true)
 |-- "property_type_ar": string (nullable = true)
 |-- "property_type_en": string (nullable = true)
 |-- "property_sub_type_id": string (nullable = true)
 |-- "property_sub_type_ar": string (nullable = true)
 |-- "property_sub_type_en": string (nullable = true)
 |-- "property_usage_ar": string (nullable = true)
 |-- "property_usage_en": string (nullable = true)
 |-- "reg_type_id": string (nullable = true)
 |-- "reg_type_ar": string (nullable = true)
 |-- "reg_type_en": string (nullable = true)
 |-- "area_id": string (nullable = true)
 |-- "area_name_ar

In [ ]:
#Basic casting for key columns
#convert important fields to correct types

from pyspark.sql.functions import col, to_date

df = df_raw

if "transaction_value" in df.columns:
    df = df.withColumn("transaction_value", col("transaction_value").cast("double"))

if "area_sqm" in df.columns:
    df = df.withColumn("area_sqm", col("area_sqm").cast("double"))

if "transaction_date" in df.columns:
    df = df.withColumn("transaction_date", to_date(col("transaction_date")))


In [ ]:
#cleaning

if "transaction_value" in df.columns:
    df = df.dropna(subset=["transaction_value"]) \
           .filter(col("transaction_value") > 0)


In [ ]:
#Show partitioning evidence

df.rdd.getNumPartitions()
df = df.repartition(200)
df.rdd.getNumPartitions()


200

In [ ]:
out_path = "/content/drive/MyDrive/dld_transactions/cleaned_parquet"
df.write.mode("overwrite").parquet(out_path)

print("Parquet saved to:", out_path)


Parquet saved to: /content/drive/MyDrive/dld_transactions/cleaned_parquet


In [ ]:
#Build the correct explicit schema (recommended)

#This cell defines a proper StructType (IDs as integers/strings, numerics as doubles, date as timestamp-ready)

from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, DoubleType
)

schema = StructType([
    StructField("transaction_id", StringType(), True),

    StructField("procedure_id", IntegerType(), True),
    StructField("trans_group_id", IntegerType(), True),
    StructField("trans_group_ar", StringType(), True),
    StructField("trans_group_en", StringType(), True),

    StructField("procedure_name_ar", StringType(), True),
    StructField("procedure_name_en", StringType(), True),

    StructField("instance_date", StringType(), True),  # parse after load

    StructField("property_type_id", IntegerType(), True),
    StructField("property_type_ar", StringType(), True),
    StructField("property_type_en", StringType(), True),

    StructField("property_sub_type_id", IntegerType(), True),
    StructField("property_sub_type_ar", StringType(), True),
    StructField("property_sub_type_en", StringType(), True),

    StructField("property_usage_ar", StringType(), True),
    StructField("property_usage_en", StringType(), True),

    StructField("reg_type_id", IntegerType(), True),
    StructField("reg_type_ar", StringType(), True),
    StructField("reg_type_en", StringType(), True),

    StructField("area_id", IntegerType(), True),
    StructField("area_name_ar", StringType(), True),
    StructField("area_name_en", StringType(), True),

    StructField("building_name_ar", StringType(), True),
    StructField("building_name_en", StringType(), True),

    StructField("project_number", StringType(), True),
    StructField("project_name_ar", StringType(), True),
    StructField("project_name_en", StringType(), True),

    StructField("master_project_en", StringType(), True),
    StructField("master_project_ar", StringType(), True),

    StructField("nearest_landmark_ar", StringType(), True),
    StructField("nearest_landmark_en", StringType(), True),

    StructField("nearest_metro_ar", StringType(), True),
    StructField("nearest_metro_en", StringType(), True),

    StructField("nearest_mall_ar", StringType(), True),
    StructField("nearest_mall_en", StringType(), True),

    StructField("rooms_ar", StringType(), True),
    StructField("rooms_en", StringType(), True),

    StructField("has_parking", IntegerType(), True),

    StructField("procedure_area", DoubleType(), True),
    StructField("actual_worth", DoubleType(), True),
    StructField("meter_sale_price", DoubleType(), True),
    StructField("rent_value", DoubleType(), True),
    StructField("meter_rent_price", DoubleType(), True),

    StructField("no_of_parties_role_1", IntegerType(), True),
    StructField("no_of_parties_role_2", IntegerType(), True),
    StructField("no_of_parties_role_3", IntegerType(), True),
])


In [ ]:
#Reload the CSV using this schema (clean, official ingestion)

csv_path = "/content/drive/MyDrive/dld_transactions/dld_transactions.csv"

df = spark.read.csv(
    csv_path,
    header=True,
    schema=schema,
    mode="DROPMALFORMED"
)


In [ ]:
#Parse the date + create time features
# parse instance_date and create year/month/day-of-week

from pyspark.sql.functions import col, to_timestamp, year, month, dayofweek

df = df.withColumn("instance_ts", to_timestamp(col("instance_date")))
df = df.withColumn("year", year(col("instance_ts"))) \
       .withColumn("month", month(col("instance_ts"))) \
       .withColumn("dow", dayofweek(col("instance_ts")))


In [ ]:
#Missing value & sanity cleaning

from pyspark.sql.functions import col

# Drop rows missing key numeric targets (pick one later; for now keep both)
df = df.dropna(subset=["actual_worth", "procedure_area"])

# Filter impossible values
df = df.filter(col("procedure_area") > 0)
df = df.filter(col("actual_worth") > 0)

# Optional: ensure parking is 0/1-ish (some datasets use null/1)
df = df.fillna({"has_parking": 0})


In [ ]:
#Partitioning evidence

print("Before repartition:", df.rdd.getNumPartitions())
df = df.repartition(200)
print("After repartition:", df.rdd.getNumPartitions())


Before repartition: 8
After repartition: 200


In [ ]:
#Save the handoff dataset as Parquet

out_path = "/content/drive/MyDrive/dld_transactions/cleaned_parquet"
df.write.mode("overwrite").parquet(out_path)
print("Saved Parquet to:", out_path)


Saved Parquet to: /content/drive/MyDrive/dld_transactions/cleaned_parquet


In [ ]:
#screenshots for ingestion

df.printSchema()
print("Row count:", df.count())
df.show(5, truncate=False)
print("Partitions:", df.rdd.getNumPartitions())


root
 |-- transaction_id: string (nullable = true)
 |-- procedure_id: integer (nullable = true)
 |-- trans_group_id: integer (nullable = true)
 |-- trans_group_ar: string (nullable = true)
 |-- trans_group_en: string (nullable = true)
 |-- procedure_name_ar: string (nullable = true)
 |-- procedure_name_en: string (nullable = true)
 |-- instance_date: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- property_type_ar: string (nullable = true)
 |-- property_type_en: string (nullable = true)
 |-- property_sub_type_id: integer (nullable = true)
 |-- property_sub_type_ar: string (nullable = true)
 |-- property_sub_type_en: string (nullable = true)
 |-- property_usage_ar: string (nullable = true)
 |-- property_usage_en: string (nullable = true)
 |-- reg_type_id: integer (nullable = true)
 |-- reg_type_ar: string (nullable = true)
 |-- reg_type_en: string (nullable = true)
 |-- area_id: integer (nullable = true)
 |-- area_name_ar: string (nullable = true)
 |-- are

In [ ]:
df.select("instance_date").show(5, truncate=False)


+-------------+
|instance_date|
+-------------+
|14-11-2013   |
|27-01-2020   |
|10-09-2014   |
|01-06-2020   |
|09-11-2016   |
+-------------+
only showing top 5 rows



In [ ]:
!ls -lh /content/drive/MyDrive/dld_transactions/cleaned_parquet


total 6.8M
-rw------- 1 root root 34K Jan 28 11:26 part-00000-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 35K Jan 28 11:26 part-00001-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 35K Jan 28 11:26 part-00002-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 34K Jan 28 11:26 part-00003-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 35K Jan 28 11:26 part-00004-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 34K Jan 28 11:26 part-00005-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 35K Jan 28 11:26 part-00006-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 35K Jan 28 11:26 part-00007-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root root 34K Jan 28 11:26 part-00008-6d0dc871-b810-4b46-bcea-9a48fda3725a-c000.snappy.parquet
-rw------- 1 root

#ML: Clustering Model

In [ ]:
# Check to see if the cleaned Parquet dataset exists in Google Drive
!ls /content/drive

MyDrive


In [ ]:
!ls /content/drive/MyDrive

'01 rec 1 test.wav'
'314 lab1.gdoc'
'AI 1Year Plan.docx'
 cleaned_parquet
'Colab Notebooks'
'Copy of Project [Python Programming]: Booking System CSIT040.gslides'
 ec48dcab-1b8d-4696-8938-0dda9cd76754.jpeg
 EResidence20120192807578.pdf
'image (1).jpg'
 image.jpg
'ML AI 1Year Plan.gdoc'
'Passport .pdf'
'Quantopian Project Audit  –  Pecha Kucha.gslides'
'Quantopian Project Audit  –  Pecha Kucha.mp4'
'Quantopian project failure analysis.gdoc'
'Quantopian project failure audit checklist.gdoc'
'Quotation_BND TO HOUSTON.pdf'
'Scanned Document (1).pdf'
'Scanned Document.pdf'
'School Diploma.pdf'
'Untitled audit checklist for Quantopian.gsheet'
'Untitled document (1).gdoc'
'Untitled document.gdoc'


In [ ]:
# Load the cleaned and preprocessed dataset produced in the ingestion phase
parquet_path = "/content/drive/MyDrive/cleaned_parquet"
df_ml = spark.read.parquet(parquet_path)

In [ ]:
# Printing the schema to understand available columns and data types
df_ml.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- procedure_id: integer (nullable = true)
 |-- trans_group_id: integer (nullable = true)
 |-- trans_group_ar: string (nullable = true)
 |-- trans_group_en: string (nullable = true)
 |-- procedure_name_ar: string (nullable = true)
 |-- procedure_name_en: string (nullable = true)
 |-- instance_date: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- property_type_ar: string (nullable = true)
 |-- property_type_en: string (nullable = true)
 |-- property_sub_type_id: integer (nullable = true)
 |-- property_sub_type_ar: string (nullable = true)
 |-- property_sub_type_en: string (nullable = true)
 |-- property_usage_ar: string (nullable = true)
 |-- property_usage_en: string (nullable = true)
 |-- reg_type_id: integer (nullable = true)
 |-- reg_type_ar: string (nullable = true)
 |-- reg_type_en: string (nullable = true)
 |-- area_id: integer (nullable = true)
 |-- area_name_ar: string (nullable = true)
 |-- are

In [ ]:
#  checking number of rows and columns

print("Rows:", df_ml.count())
print("Columns:", len(df_ml.columns))
df_ml.show(5, truncate=False)

Rows: 30173
Columns: 50
+-----------------+------------+--------------+--------------+--------------+--------------------------+-------------------------+-------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+-----------------+-----------------+-----------+----------------+-------------------+-------+--------------+------------------+--------------------------+----------------------+--------------+--------------------------+---------------------------+--------------------------+---------------------------------+---------------------------------+----------------------------+------------------------+--------------------------+---------------+------------------+--------+--------+-----------+--------------+------------+----------------+----------+----------------+--------------------+--------------------+--------------------+-----------+----+-----+----+
|transaction_id   |procedure_id|trans_group_id|trans_group_ar|tra

In [ ]:
# Checking for missing values in key numeric columns

from pyspark.sql.functions import col, sum as spark_sum, when

num_cols = ["procedure_area", "actual_worth", "meter_sale_price", "rent_value", "meter_rent_price"]
existing_num_cols = [c for c in num_cols if c in df_ml.columns]

missing = df_ml.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c + "_nulls")
    for c in existing_num_cols
])
missing.show(truncate=False)


+--------------------+------------------+----------------------+----------------+----------------------+
|procedure_area_nulls|actual_worth_nulls|meter_sale_price_nulls|rent_value_nulls|meter_rent_price_nulls|
+--------------------+------------------+----------------------+----------------+----------------------+
|0                   |0                 |0                     |0               |0                     |
+--------------------+------------------+----------------------+----------------+----------------------+



In [ ]:
# Select numeric features for clustering , these features describe property size, value, and price density

feature_cols = [
    "procedure_area",
    "actual_worth",
    "meter_sale_price",
    "meter_rent_price"
]

df_features = (
    df_ml
    .select(feature_cols)
    .filter("procedure_area > 0")
    .filter("actual_worth > 0")
)


In [ ]:
df_features.describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|    procedure_area|      actual_worth|  meter_sale_price|  meter_rent_price|
+-------+------------------+------------------+------------------+------------------+
|  count|             30173|             30173|             30173|             30173|
|   mean|146.69104928247128|1141369.1333311237|23615.968940774856|23615.968940774856|
| stddev|247.93351448611162|1179763.9031765272|  312756.464488263|  312756.464488263|
|    min|              0.09|               2.0|              0.01|              0.01|
|    max|           35703.0|           3.744E7|     1.109415556E7|     1.109415556E7|
+-------+------------------+------------------+------------------+------------------+



In [ ]:
# Combine selected features into a single feature vector

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_raw"
)

df_vector = assembler.transform(df_features)


In [ ]:
df_vector.select("features_raw").show(5, truncate=False)

+---------------------------------+
|features_raw                     |
+---------------------------------+
|[151.8,867007.0,5711.51,5711.51] |
|[76.0,201640.0,2653.16,2653.16]  |
|[356.3,1119825.0,3142.93,3142.93]|
|[159.3,581944.0,3653.13,3653.13] |
|[531.29,941510.0,1772.12,1772.12]|
+---------------------------------+
only showing top 5 rows


In [ ]:
# Scale the feature vectors so that all features contribute equally

from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=True,
    withStd=True
)

scaler_model = scaler.fit(df_vector)
df_scaled = scaler_model.transform(df_vector)

In [ ]:
df_scaled.select("features").show(5, truncate=False)

+-------------------------------------------------------------------------------------+
|features                                                                             |
+-------------------------------------------------------------------------------------+
|[0.020606131962910557,-0.23255681292875738,-0.05724728654312684,-0.05724728654312684]|
|[-0.2851209907179808,-0.7965399948251455,-0.067025981301696,-0.067025981301696]      |
|[0.8454240289054208,-0.018261393888316463,-0.06546000247915952,-0.06546000247915952] |
|[0.05085617708305152,-0.4741839717462672,-0.06382870126581834,-0.06382870126581834]  |
|[1.5512180816485495,-0.16940604199959106,-0.06984299741498903,-0.06984299741498903]  |
+-------------------------------------------------------------------------------------+
only showing top 5 rows


In [ ]:
# Train KMeans models with different K values and evaluate using Silhouette Score

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(
    featuresCol="features",
    metricName="silhouette",
    distanceMeasure="squaredEuclidean"
)

for k in [2, 3, 4, 5, 6]:
    kmeans = KMeans(k=k, seed=42, featuresCol="features")
    model = kmeans.fit(df_scaled)
    predictions = model.transform(df_scaled)
    score = evaluator.evaluate(predictions)
    print(f"K={k}, Silhouette Score={score}")

K=2, Silhouette Score=0.9956855254332777
K=3, Silhouette Score=0.7382026550611531
K=4, Silhouette Score=0.7430670629860737
K=5, Silhouette Score=0.7294057361590338
K=6, Silhouette Score=0.7284904455311111


In [ ]:
# Train final KMeans model using the best K

best_k = 4

kmeans_final = KMeans(
    k=best_k,
    seed=42,
    featuresCol="features"
)

final_model = kmeans_final.fit(df_scaled)
df_clustered = final_model.transform(df_scaled)

In [ ]:
# Analyze cluster characteristics by computing mean values per cluster, this helps interpret what each cluster represents in realworld terms

df_clustered.groupBy("prediction").mean().show(truncate=False)

+----------+-------------------+-----------------+---------------------+---------------------+---------------+
|prediction|avg(procedure_area)|avg(actual_worth)|avg(meter_sale_price)|avg(meter_rent_price)|avg(prediction)|
+----------+-------------------+-----------------+---------------------+---------------------+---------------+
|1         |0.09972222222222221|605129.0138888889|6102089.677499999    |6102089.677499999    |1.0            |
|2         |304.82915239350024 |2834415.131971893|13034.42812252964    |13034.42812252964    |2.0            |
|0         |117.52160377358473 |841054.2769905269|8371.382895169494    |8371.382895169494    |0.0            |
|3         |35703.0            |1462500.0        |40.96                |40.96                |3.0            |
+----------+-------------------+-----------------+---------------------+---------------------+---------------+



### Cluster Interpretation
By looking at the average area, total property value, and price per square meter, we can see that each cluster represents a different type of property.

Cluster 1 contains very small properties but with very high price per square meter. These are likely premium units in good locations, such as areas close to metro stations or important landmarks.

Cluster 0 represents normal residential properties with medium size and balanced prices. These properties are more common and are suitable for steady residential use or long-term investment.

Cluster 2 includes larger properties with high total value but lower price density. These are likely villas or high-end residential properties where value comes more from size than location.

Cluster 3 contains very large properties with very low price per square meter. These are likely land plots or properties in outer areas, mainly focused on long-term development rather than quick profit.



In [ ]:
# Adding readable labels to clusters for easier interpretation

from pyspark.sql.functions import when

df_labeled = df_clustered.withColumn(
    "cluster_label",
    when(df_clustered.prediction == 1, "High-density premium units")
    .when(df_clustered.prediction == 0, "Typical residential properties")
    .when(df_clustered.prediction == 2, "Large high-value properties")
    .otherwise("Large-area low-density land")
)

In [ ]:
# showing distribution of records across clusters

df_labeled.groupBy("cluster_label").count().show()

+--------------------+-----+
|       cluster_label|count|
+--------------------+-----+
|Large high-value ...| 4554|
|Typical residenti...|25546|
|High-density prem...|   72|
|Large-area low-de...|    1|
+--------------------+-----+



### Summary

The clustering results can be used to support decision making in real estate financial planning.

By grouping properties with similar characteristics, landlords and developers can quickly understand what type of property they are dealing with and what level of profitability or investment risk it may have.

For example, properties that fall into clusters with high price per square meter are likely located near metro stations or important urban landmarks. These properties may be more expensive but can generate higher returns due to demand and accessibility.

On the other hand, properties in clusters with lower price density but larger area may be more suitable for long-term development projects rather than short-term profit.

This clustering approach helps real estate agents, firms, and individual investors compare properties objectively and make better planning decisions based on data instead of assumptions.

